In [1]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

import Data.Char
import Data.Bits
import Data.Vector (Vector)
import qualified Data.Vector as V
import Data.Word
import Numeric
import Prelude hiding (lookup)

type Hash = Binary Word32
type Bitmap = Binary Word16
type Shift = Int

newtype Binary a = Binary a deriving (Enum, Ord, Real, Integral, Eq, Num, Bits, FiniteBits)

instance (FiniteBits a, Show a, Integral a) => Show (Binary a) where
    show (Binary a) = let
        str = showIntAtBase 2 intToDigit a ""
        size = finiteBitSize $ undefined `asTypeOf` a
        in replicate (size - length str) '0' ++ str

class Hashable a where
    hash :: a -> Hash

data HAMT key value
    = None
    | Leaf Hash key value
    | Many Bitmap (Vector (HAMT key value))
    deriving (Show)

empty :: HAMT k v
empty = None

In [2]:
bitsPerSubkey :: Int
bitsPerSubkey = 4

subkeyMask :: Bitmap
subkeyMask = 1 `shiftL` bitsPerSubkey - 1

subkey :: Hash -> Shift -> Int
subkey hash shift = fromIntegral $ (fromIntegral $ shiftR hash shift) .&. subkeyMask

maskIndex :: Bitmap -> Bitmap -> Int
maskIndex bitmap mask = popCount (bitmap .&. (mask - 1))

bitMask :: Hash -> Shift -> Bitmap
bitMask hash shift = 1 `shiftL` (subkey hash shift)

In [3]:
insertAt :: Vector a -> Int -> a -> Vector a
insertAt vector index a = V.take index vector V.++ V.singleton a V.++ V.drop index vector

updateAt :: Vector a -> Int -> a -> Vector a
updateAt vector index a = vector V.// [(index, a)]

deleteAt :: Vector a -> Int -> Vector a
deleteAt vector index = (V.take index vector V.++ V.drop (index+1) vector)

In [4]:
lookup :: Hashable key => key -> HAMT key value -> Maybe value
lookup key hamt = lookup' 0 (hash key) hamt

lookup' :: Shift -> Hash -> HAMT key value -> Maybe value
lookup' shift hash None = Nothing

lookup' shift hash (Leaf leafHash leafKey leafValue)
    | hash == leafHash = Just leafValue
    | otherwise = Nothing

lookup' shift hash (Many bitmap vector)
    | bitmap .&. mask == 0 = Nothing
    | otherwise = lookup' (shift+bitsPerSubkey) hash (vector V.! index)
    where
        mask = bitMask hash shift
        index = maskIndex bitmap mask

In [5]:
insert :: Hashable key => key -> value -> HAMT key value -> HAMT key value
insert key value hamt = insert' 0 (hash key) key value hamt

insert' :: Shift -> Hash -> key -> value -> HAMT key value -> HAMT key value
insert' shift hash key value None = Leaf hash key value

insert' shift hash key value leaf@(Leaf leafHash leafKey leafValue)
    | hash == leafHash = Leaf hash key value
    | otherwise = insert' shift hash key value (Many (bitMask leafHash shift) (V.singleton leaf))

insert' shift hash key value (Many bitmap vector)
    | bitmap .&. mask == 0 = let
        leaf = Leaf hash key value
        vector' = insertAt vector index leaf
        bitmap' = bitmap .|. mask
        in Many bitmap' vector'
    | otherwise = let
        subtree = vector V.! index
        subtree' = insert' (shift+bitsPerSubkey) hash key value subtree
        vector' = updateAt vector index subtree'
        in Many bitmap vector'
    where
        mask = bitMask hash shift
        index = maskIndex bitmap mask

In [6]:
delete :: Hashable key => key -> HAMT key value -> HAMT key value
delete key hamt = delete' 0 (hash key) hamt

delete' :: Shift -> Hash -> HAMT key value -> HAMT key value
delete' shift hash None = None

delete' shift hash leaf@(Leaf leafHash leafKey leafValue)
    | hash == leafHash = None
    | otherwise = leaf

delete' shift hash many@(Many bitmap vector)
    | bitmap .&. mask == 0 = many
    | otherwise = let
        subtree = vector V.! index
        subtree' = delete' (shift+bitsPerSubkey) hash subtree
        in case subtree' of
            None -> if V.length vector == 1
                then None
                else Many (bitmap .&. complement mask) (deleteAt vector index)
            Leaf{} -> if V.length vector == 1
                then subtree'
                else Many bitmap (updateAt vector index subtree')
            _ ->     Many bitmap (updateAt vector index subtree')
    where
        mask = bitMask hash shift
        index = maskIndex bitmap mask

In [7]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString as B

import Data.ByteArray.Hash

bs :: B.ByteString -> B.ByteString
bs = id

instance Hashable B.ByteString where
    hash bs = let
        FnvHash32 h = fnv1Hash bs
        in Binary h

t = insert (bs "2") 2 $ insert (bs "1") 1 empty
t' = insert (bs "3") 3 $ delete (bs "2") t
lookup (bs "1") t
delete (bs "1") t'

Just 1

Many 0001000000000000 [Leaf 00000101000011000101110100101100 "3" 3]

In [8]:
import Text.Show.Pretty

pPrint empty

None

In [9]:
t0 = insert (bs "3") 1 empty
t1 = insert (bs "30") 2 t0
t2 = insert (bs "300") 3 t1
t3 = insert (bs "3000") 4 t2

mapM_ pPrint [t0, t1, t2, t3]

Leaf 00000101000011000101110100101100 "3" 1
Many
  0001000000010000
  [ Leaf 00011110011101101010110001110100 "30" 2
  , Leaf 00000101000011000101110100101100 "3" 1
  ]
Many
  0001000000010000
  [ Leaf 00011110011101101010110001110100 "30" 2
  , Many
      0000010000000100
      [ Leaf 00000101000011000101110100101100 "3" 1
      , Leaf 01101000110100010111101010101100 "300" 3
      ]
  ]
Many
  0001000000010000
  [ Many
      1000000010000000
      [ Leaf 00011110011101101010110001110100 "30" 2
      , Leaf 10101101110001000001110011110100 "3000" 4
      ]
  , Many
      0000010000000100
      [ Leaf 00000101000011000101110100101100 "3" 1
      , Leaf 01101000110100010111101010101100 "300" 3
      ]
  ]